<a href="https://colab.research.google.com/github/aSafarpoor/AI_Project_BypedalWalker/blob/master/Edge_Blocking_Method_%2B_FreeTriAngleDegree_vs_Random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tqdm import tqdm
import numpy as np
import networkx as nx
import random
from matplotlib import pyplot as plt

In [2]:
number_of_initial_reds = 1
max_time_of_staying_red = 3

In [3]:
!wget https://raw.githubusercontent.com/aSafarpoor/storehouse/main/dataset/facebook_combined.txt

--2023-09-09 14:14:27--  https://raw.githubusercontent.com/aSafarpoor/storehouse/main/dataset/facebook_combined.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 854362 (834K) [text/plain]
Saving to: ‘facebook_combined.txt’

facebook_combined.t 100%[===================>] 834.34K  --.-KB/s    in 0.03s   

2023-09-09 14:14:27 (32.3 MB/s) - ‘facebook_combined.txt’ saved [854362/854362]



In [4]:
filename = "facebook_combined.txt"
with open(filename) as f:
    contents = f.read()

In [5]:
es = []
vs = []
es_half = []
for i in tqdm(contents.split("\n")):
    c = list(map(int,i.split()))
    try:
        es_half.append((c[0],c[1]))
        es.append((c[0],c[1]))
        es.append((c[1],c[0]))
        vs.append(c[0])
        vs.append(c[1])
    except:
        print(c)
vs = list(set(vs))

100%|██████████| 88235/88235 [00:00<00:00, 433236.42it/s]

[]


uncolored ==> 0 (blue)

spreader  ==> 1 (red)

dontcare  ==> 2 (orange)

In [6]:
colors = np.zeros(len(vs))

G = nx.Graph()
G.add_edges_from(list(set(es)))

first_run = False
if first_run:
    positions = nx.spring_layout(G)

In [7]:
am = {}
for i in tqdm(vs):
    am[i] = []
for i in tqdm(es):
    am[i[0]].append(i[1])
    am[i[1]].append(i[0])

100%|██████████| 176468/176468 [00:00<00:00, 1514748.07it/s]


In [8]:
for v in tqdm(vs):
    am[v] = list(set(am[v]))

100%|██████████| 4039/4039 [00:00<00:00, 207694.31it/s]


In [9]:
def init_color(colors,vs,number_of_initial_reds):
    init_reds = random.sample(vs,number_of_initial_reds)
    for i in init_reds:
        colors[i] = 1
    return colors

init_colors = init_color(colors,vs,number_of_initial_reds)


In [10]:
def s_function(i,j):
    x = am[i]
    y = am[j]
    A=list(set(x)|set(y)) # union
    B=list(set(x)&set(y)) # intersection
    return (len(B)+2)/len(A)

w = {}
for e in es:
    i = e[0]
    j = e[1]
    temp = s_function(i,j)
    w[(i,j)]= temp
    w[(j,i)]= temp


def p_star(w,times,am,i,j):
    try:
        p=(1-w[(i,j)]/2**(max_time_of_staying_red-times[j]))
    except:
        print(i,j,times[j])
        assert 1==0
    return p

In [46]:
def random_edges(am,probability_of_removing_edge=0.1):
    edge_removed_counter = 0
    am2 = {}
    for i in list(am.keys()):
        am2[i]=[]
    for i in list(am.keys()):
        js = am[i]
        for j in js:
            if random.random() > probability_of_removing_edge:
                am2[i].append(j)
            else:
                edge_removed_counter+=1
    print(edge_removed_counter)
    return am2

In [47]:
treeangledegreedictionart = nx.triangles(G)

In [67]:
def triangle_degree(am,probability_of_removing_edge):
    am2 = {}
    for i in list(am.keys()):
        am2[i]=[]

    templist = []
    for e in es:
        templist.append(treeangledegreedictionart[e[0]]+treeangledegreedictionart[e[1]])
    templist.sort()

    th = templist[2*int(-probability_of_removing_edge*len(templist))]
    edge_removed_counter = 0
    for i in list(am.keys()):
        js = am[i]
        for j in js:
            wij = treeangledegreedictionart[e[0]]+treeangledegreedictionart[e[1]]
            if th < wij:
                am2[i].append(j)
            else:
                edge_removed_counter+=1
    print(edge_removed_counter)
    return am2

In [68]:
def model(edge_blocking_function,colors,s,probability_of_removing_edge):
    times = np.ones(len(vs))*max_time_of_staying_red
    colors2 = np.array(colors)
    r = number_of_initial_reds
    rounds = 0
    am2 = edge_blocking_function(am,probability_of_removing_edge)

    tempsum = 0
    for v in vs:
        tempsum += len(am2[v])
    # print(tempsum)

    while(r>0):
        rounds+=1
        r = o = g = 0
        for v in vs:
            if colors[v] == 0:
                flag = False
                for n in am2[v]:
                    if colors[n]==1:
                        if random.random()<p_star(s,times,am,v,n):
                            flag=True
                if flag:
                        colors2[v]=1
                        r+=1
                else:
                    g+=1

            elif colors[v] == 1:
                if times[v]>0:
                    times[v]-=1
                    r+=1
                else:
                    colors2[v]=2
                    o+=1
            elif colors[v] == 2:
                o+=1

        colors = np.copy(colors2[:])

    return colors,rounds

In [69]:
tentimes_pstar = []
for __ in tqdm(range(10)):
    colors,rounds = model(triangle_degree,init_colors[:],w,probability_of_removing_edge=0.2)
    tentimes_pstar.append(rounds)

print("")
freetriangle_model = np.average(tentimes_pstar)
print(freetriangle_model)

 20%|██        | 2/10 [00:00<00:00,  8.06it/s]

176468
176468


 40%|████      | 4/10 [00:00<00:00,  8.36it/s]

176468
176468


 60%|██████    | 6/10 [00:00<00:00,  8.15it/s]

176468
176468


 80%|████████  | 8/10 [00:00<00:00,  8.29it/s]

176468
176468


100%|██████████| 10/10 [00:01<00:00,  8.04it/s]

176468
176468

4.0


In [70]:
tentimes_pstar = []
for __ in tqdm(range(10)):
    colors,rounds = model(random_edges,init_colors[:],w,probability_of_removing_edge=0.2)
    tentimes_pstar.append(rounds)

print("")
random_model = np.average(tentimes_pstar)
print(random_model)

 10%|█         | 1/10 [00:00<00:01,  5.84it/s]

35485
35532


 30%|███       | 3/10 [00:00<00:01,  5.96it/s]

35321
35553


 50%|█████     | 5/10 [00:00<00:00,  5.68it/s]

...

35231
35455


 90%|█████████ | 9/10 [00:01<00:00,  5.76it/s]

35018
35330


100%|██████████| 10/10 [00:01<00:00,  5.73it/s]


13.8


In [60]:
tentimes_pstar = []
for __ in tqdm(range(10)):
    colors,rounds = model(random_edges,init_colors[:],w,probability_of_removing_edge=0.2)
    tentimes_pstar.append(rounds)

print("")
random_model = np.average(tentimes_pstar)
print(random_model)

 10%|█         | 1/10 [00:00<00:01,  5.63it/s]

34877
141591
35246
141222


 30%|███       | 3/10 [00:00<00:01,  5.91it/s]

35086
141382
35339
141129


 50%|█████     | 5/10 [00:00<00:00,  6.03it/s]

...

35495
140973
35606
140862


 90%|█████████ | 9/10 [00:01<00:00,  5.94it/s]

35316
141152
35169
141299


100%|██████████| 10/10 [00:01<00:00,  5.94it/s]


13.7


In [25]:
# bit_cent = nx.betweenness_centrality(G,normalized=True)
# bval = np.array(list(bit_cent.values()))
# edge_bit_cent = nx.edge_betweenness_centrality(G, normalized=True)